In [65]:
import pandas as pd
import numpy as np
#eplus_out_file = "/home/zhiangz/Documents/HVAC-RL-Control/src/eplus-env/eplus_env/envs/eplus_models/iw_v96/learning/idf/realWeather_base_out/eplusout.csv";
eplus_out_file = '/home/zhiangz/Documents/HVAC-RL-Control/src/iw_ctrl_runs_v2/11/Eplus-env-IW-tmy3Weather-v9604-res1/Eplus-env-sub_run41/output/eplusout.csv'
occupant_count_colname = 'EMS:IW Occupy Mode Flag [](TimeStep)'
ppd_colname = 'EMS:IW Average PPD [%](TimeStep)'
energy_colname = 'EMS:IW Calculated Heating Demand [KW](TimeStep)'
stptbylogic_colname = 'EMS:IW Effective IAT Setpoint by Logics [C](TimeStep)'
iat_colname = 'EMS:IW North Zone Average Temperature [C](TimeStep)'
timestep = 5 # Minutes

In [66]:
def getFullColname(colnames, tgtname):
    for colname in colnames:
        if tgtname in colname:
            return colname;

In [67]:
df = pd.read_csv(eplus_out_file)
df_colnames = list(df.columns.values)
occupant_count_colname = getFullColname(df_colnames, occupant_count_colname)
ppd_colname = getFullColname(df_colnames, ppd_colname)
energy_colname = getFullColname(df_colnames, energy_colname)
stptbylogic_colname = getFullColname(df_colnames, stptbylogic_colname)
iat_colname = getFullColname(df_colnames, iat_colname);

In [68]:
occp_ppd = df[ppd_colname].loc[df[occupant_count_colname] > 0]
valid_energy = df[energy_colname].loc[np.isnan(df[energy_colname]) != 1]
iat_stpt_diff = df[iat_colname].loc[df[occupant_count_colname] == 0] - df[stptbylogic_colname].loc[df[occupant_count_colname] == 0]

In [69]:
mean_occp_ppd = np.mean(occp_ppd)
std_occp_ppd = np.std(occp_ppd)
total_energy = np.sum(valid_energy) * (timestep/60.0)
mean_iat_notmet = np.mean(iat_stpt_diff)
std_iat_notmet = np.std(iat_stpt_diff);

In [70]:
print ('Total HVAC energy consumption is %d kHW, mean occupied PPD is %0.02f%%, std occupied PPD is %0.02f%%'\
       ', mean IAT not met during unoccupied hour is %0.02f, std IAT not met during unoccupied hour is %0.02f.'
      %(total_energy, mean_occp_ppd, std_occp_ppd, mean_iat_notmet, std_iat_notmet))

Total HVAC energy consumption is 49303 kHW, mean occupied PPD is 8.46%, std occupied PPD is 5.31%, mean IAT not met during unoccupied hour is 1.62, std IAT not met during unoccupied hour is 2.00.


In [46]:
np.histogram(occp_ppd)

(array([9157, 1708,  538,  248,  104,   56,   43,   82,   56,   20]),
 array([  5.2219025 ,   8.5755549 ,  11.92920729,  15.28285969,
         18.63651209,  21.99016448,  25.34381688,  28.69746928,
         32.05112167,  35.40477407,  38.75842647]))